In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, date, timedelta, time
os.environ['TZ'] ='America/New_York'

#import yfinance as yf

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

from ib_async import *
util.startLoop()

### Data Prep for Inferencing

In [2]:
ib = IB()
ib.connect(port=7496, clientId=3)

<IB connected to 127.0.0.1:7496 clientId=3>

Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IBKR and Trader Workstation has been restored - data maintained. All data farms are connected: hfarm; usfarm.nj; cashfarm; usopt; usfarm; ushmds; secdefil.


In [3]:
contract = Contract(symbol='AAPL', secType='STK', exchange='SMART', currency='USD')
ib.qualifyContracts(contract)

[Contract(secType='STK', conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS')]

In [4]:
contract_vix = Contract(symbol= 'VIX', secType = 'IND',exchange = 'CBOE', currency='USD')
ib.qualifyContracts(contract_vix)

[Contract(secType='IND', conId=13455763, symbol='VIX', exchange='CBOE', currency='USD', localSymbol='VIX')]

In [5]:
contract_nas = Contract(symbol= 'TQQQ', secType = 'STK',exchange = 'SMART', currency='USD')
ib.qualifyContracts(contract_nas)

[Contract(secType='STK', conId=72539702, symbol='TQQQ', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='TQQQ', tradingClass='NMS')]

In [6]:
def get_fridays_in_range(start_date, end_date):
    """
    Returns a list of dates that are Fridays within the given date range.

    Args:
        start_date (date): The start date of the range.
        end_date (date): The end date of the range.

    Returns:
        list: A list of date objects representing Fridays within the range.
    """
    fridays = []
    all_dates = []
    current_date = start_date
    all_current_date = start_date

    # Adjust to the first Friday
    days_until_friday = (4 - current_date.weekday()) % 7
    current_date += timedelta(days=days_until_friday)

    while current_date <= end_date:
        fridays.append(current_date)
        current_date += timedelta(days=7) # Move to the next Friday

    while all_current_date <= end_date:
        all_dates.append(all_current_date)
        all_current_date += timedelta(days=1)

    biz_dates = [ d for d in all_dates if d.weekday()<5]
    
    return fridays, biz_dates

# Example Usage
start_date = date(2024, 2, 1)
end_date = date(2025, 4, 4)

fridays, all_dates = get_fridays_in_range(start_date, end_date)
# removing Non-trading Holidays

fridays.remove(date(2024, 3, 29))
fridays.remove(date(2024, 11, 29))

#all_dates.remove(date(2024, 1, 15))
all_dates.remove(date(2024, 2, 19))
all_dates.remove(date(2024, 3, 29))
all_dates.remove(date(2024, 5, 27))
all_dates.remove(date(2024, 6, 19))
all_dates.remove(date(2024, 7, 3))
all_dates.remove(date(2024, 7, 4))
all_dates.remove(date(2024, 9, 2))
all_dates.remove(date(2024, 11, 28))
all_dates.remove(date(2024, 11, 29))
all_dates.remove(date(2024, 12, 24))
all_dates.remove(date(2024, 12, 25))

all_dates.remove(date(2025, 1, 1))
all_dates.remove(date(2025, 1, 8))
all_dates.remove(date(2025, 1, 20))
all_dates.remove(date(2025, 2, 17))


In [7]:
# duration is fixed for an year
def simple_data_req( contract, end_date , barsize ="1 min", data_clean = True):

   
    df_acc=pd.DataFrame()
    df_hist=pd.DataFrame()

    for i in range(17):
        print(f'fetching data for {i}')
        bars = ib.reqHistoricalData(contract= contract, endDateTime=end_date, barSizeSetting= barsize, durationStr= "20 D", whatToShow="TRADES", useRTH=True)
        df = util.df(bars)
        end_date = end_date - timedelta(20+2)
        df_acc = pd.concat([df, df_acc], ignore_index=True)
        print(f"head date {df_acc.head(1)['date']}")
        print(f"tail date {df_acc.tail(1)['date']}")

    df_acc['Date_Only'] = df_acc['date'].dt.date
    
    if data_clean == True:
        df_hist = df_acc[(df_acc['date'].dt.time >= time(9, 30)) & (df_acc['date'].dt.time < time(16, 00))]
    else:
        df_hist = df_acc

    #df_hist.drop_duplicates(inplace=True)
    
    return df_hist

In [8]:
# picking out Friday and before specific-time data

#specific_time = time(14, 30, 0)
def data_filter_v46(df_initial, list_days, specific_time_before = None, specific_time_after= None, on_time= None):
    his_df = pd.DataFrame()

    df_trimmed = df_initial[df_initial['date'].dt.date.isin(list_days)]

    if specific_time_before is not None:

        his_df = df_trimmed[(df_trimmed['date'].dt.time < specific_time_before)]
        
            
    if specific_time_after is not None:

        his_df = df_trimmed[(df_trimmed['date'].dt.time >= specific_time_after)]

    if on_time is not None:
        his_df = df_trimmed[(df_trimmed['date'].dt.time == on_time)]

    if (specific_time_before is None and specific_time_after is None) and on_time is None:

        his_df = df_trimmed
        
    return his_df

In [9]:
from datetime import date, timedelta

def calculate_business_days(start_date, end_date):
    """
    Calculates the number of business days between two dates, excluding weekends.

    Args:
        start_date (date): The start date.
        end_date (date): The end date.

    Returns:
        int: The number of business days.
    """
    business_days = 0
    current_date = start_date
    while current_date <= end_date:
        if current_date.weekday() < 5:  # Monday to Friday
            business_days += 1
        current_date += timedelta(days=1)
    return business_days

# Example usage:
# start_date = date(2025, 2, 1)  # April 7, 2025 (Monday)
# end_date = date(2025, 4, 3)  # April 18, 2025 (Friday)
# num_business_days = calculate_business_days(start_date, end_date)
# print(f"Number of business days between {start_date} and {end_date}: {num_business_days}")

### Trainning Data Set Collection ###

In [10]:
barsize ="1 min"

df_all_days_stock = simple_data_req(contract = contract, end_date = end_date, barsize= barsize )

fetching data for 0
head date 0   2025-03-10 09:30:00-04:00
Name: date, dtype: datetime64[ns, US/Eastern]
tail date 7799   2025-04-04 15:59:00-04:00
Name: date, dtype: datetime64[ns, US/Eastern]
fetching data for 1
head date 0   2025-02-13 09:30:00-05:00
Name: date, dtype: datetime64[ns, US/Eastern]
tail date 15599   2025-04-04 15:59:00-04:00
Name: date, dtype: datetime64[ns, US/Eastern]
fetching data for 2
head date 0   2025-01-22 09:30:00-05:00
Name: date, dtype: datetime64[ns, US/Eastern]
tail date 23399   2025-04-04 15:59:00-04:00
Name: date, dtype: datetime64[ns, US/Eastern]
fetching data for 3
head date 0   2024-12-27 09:30:00-05:00
Name: date, dtype: datetime64[ns, US/Eastern]
tail date 31199   2025-04-04 15:59:00-04:00
Name: date, dtype: datetime64[ns, US/Eastern]
fetching data for 4
head date 0   2024-12-06 09:30:00-05:00
Name: date, dtype: datetime64[ns, US/Eastern]
tail date 38819   2025-04-04 15:59:00-04:00
Name: date, dtype: datetime64[ns, US/Eastern]
fetching data for 5
h

In [ ]:
for i in df_all_days_stock['Date_Only'].unique():
    if df_all_days_stock[df_all_days_stock['Date_Only'] ==i].shape[0] % 390 != 0:
                print(i)

In [ ]:
df_VIX = simple_data_req(contract = contract_vix, end_date = end_date, barsize= barsize )


In [ ]:
df_TQQQ = simple_data_req(contract = contract_nas, end_date = end_date, barsize= barsize )

In [11]:
### data cleaning ## step
# clean duplicate
# is in the days of interest
# within windows

def data_cleaning(df, in_days):
    df_his = pd.DataFrame()
    df_return = pd.DataFrame()
    
    df.drop_duplicates(inplace = True)
    df_his = df[df["Date_Only"].isin(in_days)]
    df_return = df_his[(df_his['date'].dt.time > time(9,29)) & (df_his['date'].dt.time < time(16,00))]

    for i in df_return['Date_Only'].unique():
        if df_return[df_return['Date_Only'] ==i].shape[0] % 390 != 0:
            print(i)
    
    return df_return

In [12]:
df_AAPL = data_cleaning(df_all_days_stock, all_dates)

df_AAPL.shape

(100620, 9)

In [ ]:
df_TQQQ_trim = data_cleaning(df_TQQQ, all_dates)

df_TQQQ_trim.shape

In [ ]:
df_VIX_trim = data_cleaning(df_VIX, all_dates)

df_VIX_trim.shape

In [13]:
df_AAPL.to_csv('AAPL_master_stock_info.csv', index=False)
df_AAPL.to_pickle('AAPL_master_stock_info.pkl')

#df_VIX_trim.to_csv('VIX_master_info.csv', index=False)
#df_VIX_trim.to_pickle('VIX_master_info.pkl')

#df_TQQQ_trim.to_csv('TQQQ_master_info.csv', index=False)
#df_TQQQ_trim.to_pickle('TQQQ_master_info.pkl')

In [14]:
df_AAPL = pd.read_pickle('AAPL_master_stock_info.pkl')
df_VIX = pd.read_pickle('VIX_master_info.pkl')
df_TQQQ = pd.read_pickle('TQQQ_master_info.pkl')

In [15]:
cut_time = time(13, 30, 0)

In [16]:

def data_sparcing_construc(df, sp_day, *, cut_time = None):
    #df_group = df.groupby("Date_Only").std(numeric_only=True)
    df_filter_prior = data_filter_v46(df, sp_day, specific_time_before = cut_time )
    df_group_prior = df_filter_prior.groupby("Date_Only").std(numeric_only=True)
    
    df_filter_after = data_filter_v46(df, sp_day, specific_time_after = cut_time )
    df_group_after = df_filter_after.groupby("Date_Only").std(numeric_only=True)


    df_group_train =pd.DataFrame()
    df_group_train['Target_std'] = pd.DataFrame(df_group_after[['open','high','low','close']].max(axis=1))
    df_group_train['Target_spread'] = df_filter_after.groupby('Date_Only')['open'].max() - df_filter_after.groupby('Date_Only')['open'].min()
    
    df_group_train['prior_std'] = pd.DataFrame(df_group_prior[['open','high','low','close']].max(axis=1))
    df_group_train['prior_Vol'] = pd.DataFrame(df_filter_prior.groupby("Date_Only").mean()['volume'])
    df_group_train['prior_spread'] = df_filter_prior.groupby('Date_Only')['open'].max() - df_filter_prior.groupby('Date_Only')['open'].min()

    df_group_train['prior_range'] =pd.DataFrame(df_filter_prior.groupby('Date_Only')['close'].max() -df_filter_prior.groupby('Date_Only')['close'].min() )
    
    df_initial = df_filter_prior.copy()
    new_time = time(cut_time.hour, cut_time.minute -1)
    df_group_train['cut_close'] = df_filter_prior[(df_filter_prior['date'].dt.time == new_time)]['close'].values

    cut_name =['cut_1hr','cut_2hr','cut_3hr']
    for i in range(3):
        print(i)
        prior = time(cut_time.hour - (i+1), cut_time.minute)
        df_group_train[cut_name[i]] = df_initial[(df_initial['date'].dt.time == new_time)]['open'].values- df_initial[(df_initial['date'].dt.time == prior)]['open'].values

    
    return df_group_train, df_filter_prior, df_filter_after, df_group_prior, df_group_after
    

In [17]:
df_group_train, df_filter_prior, df_filter_after, df_group_prior, df_group_after = data_sparcing_construc(df_AAPL, all_dates, cut_time= cut_time)

0
1
2


In [ ]:
# df_VIX_check = df_all_days_VIX[(df_all_days_VIX['date'].dt.time > time(9,29)) & (df_all_days_VIX['date'].dt.time < time(16,00))]

In [ ]:
# for d in biz_dates:
#     for df in df_TQQQ_clean:
#         if len(df_TQQQ_clean[df_TQQQ_clean['Date_Only']==d]) !=390:
#             print(d)

In [ ]:
# # switching
# df_VIX = pd.DataFrame()
# df_VIX = df_VIX_clean.copy()

# df_TQQQ = pd.DataFrame()
# df_TQQQ = df_TQQQ_clean.copy()

# df_all_days_stock = pd.DataFrame()
# df_all_days_stock = df_stock_clean.copy()

In [18]:
df_group_train_vix, *_  = data_sparcing_construc(df_VIX, all_dates, cut_time= cut_time)
df_group_train_tqqq, *_ = data_sparcing_construc(df_TQQQ, all_dates, cut_time= cut_time)

0
1
2
0
1
2


In [19]:
df_final_jointed_vix=pd.DataFrame()
df_final_jointed_all=pd.DataFrame()

df_group_train_vix.columns =['vix_std','vix_spread','vix_prior_std','vix_Vol', 'vix_pr_spread','vix_prior_range','vix_cut_open','vix_cut_1hr','vix_cut_2hr','vix_cut_3hr']
df_final_jointed_vix = pd.concat([df_group_train,df_group_train_vix], axis=1)
df_group_train_tqqq.columns =['tqqq_std','tqqq_spread','tqqq_prior_std','tqqq_Vol','tqqq_pr_spread','tqqq_prior_range','tqqq_cut_open','tqqq_cut_1hr','tqqq_cut_2hr','tqqq_cut_3hr']
df_final_jointed_all = pd.concat([df_final_jointed_vix,df_group_train_tqqq], axis=1)
df_final_jointed_all.drop(['vix_std','vix_Vol','tqqq_std'], axis=1, inplace=True)

In [20]:
df_final_jointed_all

,Target_std,Target_spread,prior_std,prior_Vol,prior_spread,prior_range,cut_close,cut_1hr,cut_2hr,cut_3hr,...,vix_cut_3hr,tqqq_spread,tqqq_prior_std,tqqq_Vol,tqqq_pr_spread,tqqq_prior_range,tqqq_cut_open,tqqq_cut_1hr,tqqq_cut_2hr,tqqq_cut_3hr
Date_Only,,,,,,,,,,,,,,,,,,,,,
2024-03-20,0.734022,2.15,0.372571,64240.650000,1.63,1.62,176.33,0.01,0.33,0.22,...,-0.45,2.41,0.122997,91511.145833,0.79,0.80,59.53,-0.09,-0.15,-0.26
2024-03-21,0.315053,1.23,1.684435,218968.054167,5.54,5.52,172.11,-0.34,-0.59,-2.80,...,0.09,0.99,0.215014,161321.120833,0.86,0.86,63.52,-0.38,0.09,0.05
2024-03-22,0.184713,0.76,0.657411,130384.545833,2.83,2.84,172.61,0.17,0.27,0.74,...,-0.12,0.73,0.252336,101553.170833,1.04,1.04,62.89,0.40,0.41,0.62
2024-03-25,0.163684,1.06,0.440192,98426.495833,1.92,1.92,171.37,0.64,1.26,1.34,...,-0.02,0.62,0.272103,107692.295833,1.27,1.28,62.35,0.22,0.17,0.33
2024-03-26,0.222926,1.17,0.290437,90982.791667,1.55,1.57,170.82,-0.36,0.21,0.15,...,-0.08,1.23,0.172755,105764.204167,0.74,0.75,62.40,-0.14,-0.23,-0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-31,0.976527,4.72,1.121809,77499.995833,4.57,4.55,219.87,-1.33,0.47,0.97,...,-0.69,2.44,0.667997,249690.791667,2.83,2.81,55.42,-0.46,0.39,1.52
2025-04-01,0.485632,2.07,0.764383,55328.245833,4.26,4.23,222.34,-0.32,-0.63,0.75,...,0.71,1.98,0.866048,244544.458333,3.24,3.25,57.95,-0.30,-0.37,1.67
2025-04-02,0.459471,1.80,0.785839,49057.783333,3.92,3.93,224.72,0.80,0.46,0.90,...,0.47,2.00,1.026990,243414.945833,4.97,4.95,60.68,0.74,1.40,2.28


In [21]:
df_final_jointed_all.columns

Index(['Target_std', 'Target_spread', 'prior_std', 'prior_Vol', 'prior_spread',
       'prior_range', 'cut_close', 'cut_1hr', 'cut_2hr', 'cut_3hr',
       'vix_spread', 'vix_prior_std', 'vix_pr_spread', 'vix_prior_range',
       'vix_cut_open', 'vix_cut_1hr', 'vix_cut_2hr', 'vix_cut_3hr',
       'tqqq_spread', 'tqqq_prior_std', 'tqqq_Vol', 'tqqq_pr_spread',
       'tqqq_prior_range', 'tqqq_cut_open', 'tqqq_cut_1hr', 'tqqq_cut_2hr',
       'tqqq_cut_3hr'],
      dtype='object')

In [22]:
df_final_jointed_all.drop(['vix_spread','tqqq_spread'], axis=1, inplace =True)

In [23]:
df_final_jointed_all.columns

Index(['Target_std', 'Target_spread', 'prior_std', 'prior_Vol', 'prior_spread',
       'prior_range', 'cut_close', 'cut_1hr', 'cut_2hr', 'cut_3hr',
       'vix_prior_std', 'vix_pr_spread', 'vix_prior_range', 'vix_cut_open',
       'vix_cut_1hr', 'vix_cut_2hr', 'vix_cut_3hr', 'tqqq_prior_std',
       'tqqq_Vol', 'tqqq_pr_spread', 'tqqq_prior_range', 'tqqq_cut_open',
       'tqqq_cut_1hr', 'tqqq_cut_2hr', 'tqqq_cut_3hr'],
      dtype='object')

In [24]:
df_final_jointed_all.to_pickle('AAPL_trainning_master_data.pkl')